In [1]:
from langchain.agents import create_spark_sql_agent
from langchain_community.agent_toolkits import SparkSQLToolkit
from langchain_community.utilities.spark_sql import SparkSQL
from pyspark.sql import SparkSession


Carichiamo il nostro file csv

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
schema = "progetto_big_data"
spark.sql(f"CREATE DATABASE IF NOT EXISTS {schema}")
spark.sql(f"USE {schema}")
csv_file_path = "/Users/danielepica/Desktop/Big_Data/progetto_personale/LangChain/Sales Transaction.csv"
table = "selling"
spark.read.csv(csv_file_path, header=True, inferSchema=True).write.saveAsTable(table)
spark.table(table).show()

24/07/22 11:49:39 WARN Utils: Your hostname, MacBook-Pro-di-Daniele.local resolves to a loopback address: 127.0.0.1; using 192.168.0.103 instead (on interface en0)
24/07/22 11:49:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/22 11:49:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/22 11:49:45 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
24/07/22 11:49:45 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
24/07/22 11:49:45 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
24/07/22 

+-------------+---------+---------+--------------------+-----+--------+----------+--------------+
|TransactionNo|     Date|ProductNo|         ProductName|Price|Quantity|CustomerNo|       Country|
+-------------+---------+---------+--------------------+-----+--------+----------+--------------+
|       558533|6/30/2019|   82494L|Wooden Frame Anti...|13.27|       6|     16717|United Kingdom|
|       558533|6/30/2019|    84755|Colour Glass T-Li...|10.92|      16|     16717|United Kingdom|
|       558533|6/30/2019|    84949|Silver Hanging T-...|11.94|       6|     16717|United Kingdom|
|       558533|6/30/2019|   85123A|Cream Hanging Hea...|13.27|       6|     16717|United Kingdom|
|       558534|6/30/2019|    23025|Drawer Knob Vinta...|12.38|      12|     13988|United Kingdom|
|       558534|6/30/2019|    23032|Drawer Knob Crack...|11.94|       6|     13988|United Kingdom|
|       558534|6/30/2019|    21932|Scandinavian Pais...|11.94|      10|     13988|United Kingdom|
|       558534|6/30/

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

chat = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
    api_key="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" # Optional if not set as an environment variable
)


In [4]:
spark_sql = SparkSQL(schema=schema)

In [5]:
from langchain.agents import AgentType

In [6]:
spark_sql = SparkSQL(schema=schema)
toolkit = SparkSQLToolkit(db=spark_sql, llm=chat)
agent_executor = create_spark_sql_agent(llm=chat, toolkit=toolkit, verbose=True, handle_parsing_errors=True)

In [9]:
agent_executor.invoke("Qual è il prodotto più acquistato?")



> Entering new AgentExecutor chain...
Thought: I should look at the tables in the database to see what I can query.
Action: list_tables_sql_db
Action Input: 
Observation: selling
Thought:I have a list of tables, and I see that there is a table called "selling". This could be relevant to the question about the most purchased product.
Action: schema_sql_db
Action Input: selling
Observation: CREATE TABLE spark_catalog.progetto_big_data.selling (
  TransactionNo STRING,
  Date STRING,
  ProductNo STRING,
  ProductName STRING,
  Price DOUBLE,
  Quantity INT,
  CustomerNo STRING,
  Country STRING)
;

/*
3 rows from selling table:
TransactionNo	Date	ProductNo	ProductName	Price	Quantity	CustomerNo	Country
558533	6/30/2019	82494L	Wooden Frame Antique White	13.27	6	16717	United Kingdom
558533	6/30/2019	84755	Colour Glass T-Light Holder Hanging	10.92	16	16717	United Kingdom
558533	6/30/2019	84949	Silver Hanging T-Light Holder	11.94	6	16717	United Kingdom
*/
Thought:Thought: I have the schema of

{'input': 'Qual è il prodotto più acquistato?',
 'output': 'Il prodotto più acquistato è il "Popcorn Holder".'}